In [22]:
import cv2
import numpy as np
# load
photo_number = 12565

def convertScale(img, alpha, beta):
    """Add bias and gain to an image with saturation arithmetics. Unlike
    cv2.convertScaleAbs, it does not take an absolute value, which would lead to
    nonsensical results (e.g., a pixel at 44 with alpha = 3 and beta = -210
    becomes 78 with OpenCV, when in fact it should become 0).
    """

    new_img = img * alpha + beta
    new_img[new_img < 0] = 0
    new_img[new_img > 255] = 255
    return new_img.astype(np.uint8)

# Automatic brightness and contrast optimization with optional histogram clipping
def automatic_brightness_and_contrast(image, clip_hist_percent=25):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Calculate grayscale histogram
    hist = cv2.calcHist([gray],[0],None,[256],[0,256])
    hist_size = len(hist)

    # Calculate cumulative distribution from the histogram
    accumulator = []
    accumulator.append(float(hist[0]))
    for index in range(1, hist_size):
        accumulator.append(accumulator[index -1] + float(hist[index]))

    # Locate points to clip
    maximum = accumulator[-1]
    clip_hist_percent *= (maximum/100.0)
    clip_hist_percent /= 2.0

    # Locate left cut
    minimum_gray = 0
    while accumulator[minimum_gray] < clip_hist_percent:
        minimum_gray += 1

    # Locate right cut
    maximum_gray = hist_size -1
    while accumulator[maximum_gray] >= (maximum - clip_hist_percent):
            maximum_gray -= 1

    # Calculate alpha and beta values
    if maximum_gray - minimum_gray == 0:
        alpha = 255
    else:
        alpha = 255 / (maximum_gray - minimum_gray)
    beta = -minimum_gray * alpha

    '''
    # Calculate new histogram with desired range and show histogram 
    new_hist = cv2.calcHist([gray],[0],None,[256],[minimum_gray,maximum_gray])
    plt.plot(hist)
    plt.plot(new_hist)
    plt.xlim([0,256])
    plt.show()
    '''

    auto_result = convertScale(image, alpha=alpha, beta=beta)
    return (auto_result, alpha, beta)

for i in range(7216,photo_number+1):
    img = cv2.imread('./example_images/ ('+str(i)+').jpg')
    
    auto_result, alpha, beta = automatic_brightness_and_contrast(img)
    # 水平翻转
    flip_horiz_img = cv2.flip(img, 1)
    # 垂直翻转
    #flip_verti_img = cv2.flip(img, 0)
    # 水平垂直翻转
    #flip_horandver_img = cv2.flip(img, -1)
    
    cv2.imwrite('./example_images/FHI('+str(i)+').jpg',flip_horiz_img)
    cv2.imwrite('./example_images/Light('+str(i)+').jpg', auto_result)
    
    img = cv2.imread('./example_images/FHI('+str(i)+').jpg')
    auto_result, alpha, beta = automatic_brightness_and_contrast(img)
    cv2.imwrite('./example_images/FHI_Light('+str(i)+').jpg',auto_result)


In [23]:
import os
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import project_utils
# from datasets.get_test_image_list import get_test_list, get_pdefined_anchors, getImagePath
from py_utils import dir_utils, load_utils


def viewBBoxes(image_file, bboxes, titles, showImageName=True):

    n_items_per_row = 4
    image = Image.open(image_file)
    image = np.array(image, dtype=np.uint8)
    n_crops = len(bboxes)
    n_rows = n_crops // n_items_per_row + 1

    fig = plt.figure(figsize=[20, 20])
    if showImageName:
        fig.suptitle(os.path.basename(image_file))

    for idx, s_bbox in enumerate(bboxes):
        ax =fig.add_subplot(n_rows, n_items_per_row, idx+1)
        ax.imshow(image)
        ax.set_axis_off()

        ax.set_title(titles[idx])

        rect_i = patches.Rectangle((s_bbox[0], s_bbox[1]), s_bbox[2]-s_bbox[0], s_bbox[3]-s_bbox[1], 
                                   linewidth=2, edgecolor='yellow', facecolor='none')
        # Add the patch to the Axes
        ax.add_patch(rect_i)
    plt.show(block=False)
    #raw_input("Press Enter to continue...")
    #plt.close()



annotation_path = 'ProposalResults/ViewProposalResults-tmp.txt'
image_path_root = 'example_images/'

image_data = load_utils.load_json(annotation_path)
image_name_list = sorted(image_data)

names = []
marks = []
bbboxes = []
light_flip = 0
normal = 0
light = 0
flip = 0
photo_number = 12565
for idx, image_name in enumerate(image_name_list):
    #if idx<20:4444444
    #    continue
    
    bboxes = image_data[image_name]['bboxes']
    scores = image_data[image_name]['scores']
    names.append(image_name)
    bbboxes.append(bboxes)
    marks.append(scores)

for i in range(photo_number):
    if marks[i]<marks[i+photo_number] and marks[i+photo_number*2]<marks[i+photo_number] and marks[i+photo_number*3]<marks[i+photo_number]:
        flip+=1
    elif marks[i+photo_number]<marks[i] and marks[i+photo_number*2] < marks[i] and marks[i+photo_number*3]<marks[i]:
        normal+=1
    elif marks[i]<marks[i+photo_number*2] and marks[i+photo_number] < marks[i+photo_number*2] and marks[i+photo_number*3]<marks[i+photo_number*2]:
        light+=1
    elif marks[i]<marks[i+photo_number*3] and marks[i+photo_number] < marks[i+photo_number*3] and marks[i+photo_number*2]<marks[i+photo_number*3]:
        light_flip+=1
        
print(normal,light,flip,light_flip)
    
for i in range(photo_number):
    if marks[i]<marks[i+photo_number] and marks[i+photo_number*2]<marks[i+photo_number] and marks[i+photo_number*3]<marks[i+photo_number]:
        bboxes = bbboxes[i+photo_number]
        scores = marks[i+photo_number]
    
        img = Image.open("example_images/"+names[i+photo_number])
        for j in range(5):
            cropped = img.crop(bbboxes[i+photo_number][j])
            cropped.save("result_image/cv_cut"+str(j)+names[i+photo_number])
        s_image_path = os.path.join(image_path_root, names[i+photo_number])
        viewBBoxes(s_image_path, bboxes, scores)
        
        if marks[i+photo_number*2]<marks[i] and marks[i+photo_number*3]<marks[i]:
            bboxes = bbboxes[i]
            scores = marks[i]
            
            img = Image.open("example_images/"+names[i])
            for k in range(5):
                cropped = img.crop(bbboxes[i][k])
                cropped.save("result_image/cv_scut"+str(k)+names[i])
            s_image_path = os.path.join(image_path_root, names[i])
            viewBBoxes(s_image_path, bboxes, scores)
                             
        elif marks[i]<marks[i+photo_number*2] and marks[i+photo_number*3]<marks[i+photo_number*2]:
            bboxes = bbboxes[i+photo_number*2]
            scores = marks[i+photo_number*2]
                             
            img = Image.open("example_images/"+names[i+photo_number*2])
            for k in range(5):
                cropped = img.crop(bbboxes[i+photo_number*2][k])
                cropped.save("result_image/cv_scut"+str(k)+names[i+photo_number*2])
            s_image_path = os.path.join(image_path_root, names[i+photo_number*2])
            viewBBoxes(s_image_path, bboxes, scores)
                             
        elif marks[i]<marks[i+photo_number*3] and marks[i+photo_number*2]<marks[i+photo_number*3]:
            bboxes = bbboxes[i+photo_number*3]
            scores = marks[i+photo_number*3]
                             
            img = Image.open("example_images/"+names[i+photo_number*3])
            for k in range(5):
                cropped = img.crop(bbboxes[i+photo_number*3][k])
                cropped.save("result_image/cv_scut"+str(k)+names[i+photo_number*3])
            s_image_path = os.path.join(image_path_root, names[i+photo_number*3])
            viewBBoxes(s_image_path, bboxes, scores)
                             
    elif marks[i+photo_number]<marks[i] and marks[i+photo_number*2] < marks[i] and marks[i+photo_number*3]<marks[i]:
        bboxes = bbboxes[i]
        scores = marks[i]
    
        img = Image.open("example_images/"+names[i])
        for j in range(5):
            cropped = img.crop(bbboxes[i][j])
            cropped.save("result_image/cv_cut"+str(j)+names[i])
        s_image_path = os.path.join(image_path_root, names[i])
        viewBBoxes(s_image_path, bboxes, scores)
                             
        if marks[i+photo_number*2]<marks[i+photo_number] and marks[i+photo_number*3]<marks[i+photo_number]:
            bboxes = bbboxes[i+photo_number]
            scores = marks[i+photo_number]
            
            img = Image.open("example_images/"+names[i+photo_number])
            for k in range(5):
                cropped = img.crop(bbboxes[i+photo_number][k])
                cropped.save("result_image/cv_scut"+str(k)+names[i+photo_number])
            s_image_path = os.path.join(image_path_root, names[i+photo_number])
            viewBBoxes(s_image_path, bboxes, scores)
                             
        elif marks[i+photo_number]<marks[i+photo_number*2] and marks[i+photo_number*3]<marks[i+photo_number*2]:
            bboxes = bbboxes[i+photo_number*2]
            scores = marks[i+photo_number*2]
                             
            img = Image.open("example_images/"+names[i+photo_number*2])
            for k in range(5):
                cropped = img.crop(bbboxes[i+photo_number*2][k])
                cropped.save("result_image/cv_scut"+str(k)+names[i+photo_number*2])
            s_image_path = os.path.join(image_path_root, names[i+photo_number*2])
            viewBBoxes(s_image_path, bboxes, scores)
                             
        elif marks[i+photo_number]<marks[i+photo_number*3] and marks[i+photo_number*2]<marks[i+photo_number*3]:
            bboxes = bbboxes[i+photo_number*3]
            scores = marks[i+photo_number*3]
                             
            img = Image.open("example_images/"+names[i+photo_number*3])
            for k in range(5):
                cropped = img.crop(bbboxes[i+photo_number*3][k])
                cropped.save("result_image/cv_scut"+str(k)+names[i+photo_number*3])
            s_image_path = os.path.join(image_path_root, names[i+photo_number*3])
            viewBBoxes(s_image_path, bboxes, scores)
                             
    elif marks[i]<marks[i+photo_number*2] and marks[i+photo_number] < marks[i+photo_number*2] and marks[i+photo_number*3]<marks[i+photo_number*2]:
        bboxes = bbboxes[i+photo_number*2]
        scores = marks[i+photo_number*2]
    
        img = Image.open("example_images/"+names[i+photo_number*2])
        for j in range(5):
            cropped = img.crop(bbboxes[i+photo_number*2][j])
            cropped.save("result_image/cv_cut"+str(j)+names[i+photo_number*2])
        s_image_path = os.path.join(image_path_root, names[i+photo_number*2])
        viewBBoxes(s_image_path, bboxes, scores)
                             
        if marks[i+photo_number]<marks[i] and marks[i+photo_number*3]<marks[i]:
            bboxes = bbboxes[i]
            scores = marks[i]
            
            img = Image.open("example_images/"+names[i])
            for k in range(5):
                cropped = img.crop(bbboxes[i][k])
                cropped.save("result_image/cv_scut"+str(k)+names[i])
            s_image_path = os.path.join(image_path_root, names[i])
            viewBBoxes(s_image_path, bboxes, scores)
                             
        elif marks[i]<marks[i+photo_number] and marks[i+photo_number*3]<marks[i+photo_number]:
            bboxes = bbboxes[i+photo_number]
            scores = marks[i+photo_number]
                             
            img = Image.open("example_images/"+names[i+photo_number])
            for k in range(5):
                cropped = img.crop(bbboxes[i+photo_number][k])
                cropped.save("result_image/cv_scut"+str(k)+names[i+photo_number])
            s_image_path = os.path.join(image_path_root, names[i+photo_number])
            viewBBoxes(s_image_path, bboxes, scores)
                             
        elif marks[i+photo_number]<marks[i+photo_number*3] and marks[i]<marks[i+photo_number*3]:
            bboxes = bbboxes[i+photo_number*3]
            scores = marks[i+photo_number*3]
                             
            img = Image.open("example_images/"+names[i+photo_number*3])
            for k in range(5):
                cropped = img.crop(bbboxes[i+photo_number*3][k])
                cropped.save("result_image/cv_scut"+str(k)+names[i+photo_number*3])
            s_image_path = os.path.join(image_path_root, names[i+photo_number*3])
            viewBBoxes(s_image_path, bboxes, scores)
                             
    elif marks[i]<marks[i+photo_number*3] and marks[i+photo_number] < marks[i+photo_number*3] and marks[i+photo_number*2]<marks[i+photo_number*3]:
        bboxes = bbboxes[i+photo_number*3]
        scores = marks[i+photo_number*3]
    
        img = Image.open("example_images/"+names[i+photo_number*3])
        for j in range(5):
            cropped = img.crop(bbboxes[i+photo_number*3][j])
            cropped.save("result_image/cv_cut"+str(j)+names[i+photo_number*3])
        s_image_path = os.path.join(image_path_root, names[i+photo_number*3])
        viewBBoxes(s_image_path, bboxes, scores)
                             
        if marks[i+photo_number*2]<marks[i] and marks[i+photo_number]<marks[i]:
            bboxes = bbboxes[i]
            scores = marks[i]
            
            img = Image.open("example_images/"+names[i])
            for k in range(5):
                cropped = img.crop(bbboxes[i][k])
                cropped.save("result_image/cv_scut"+str(k)+names[i])
            s_image_path = os.path.join(image_path_root, names[i])
            viewBBoxes(s_image_path, bboxes, scores)
                             
        elif marks[i]<marks[i+photo_number*2] and marks[i+photo_number]<marks[i+photo_number*2]:
            bboxes = bbboxes[i+photo_number*2]
            scores = marks[i+photo_number*2]
                             
            img = Image.open("example_images/"+names[i+photo_number*2])
            for k in range(5):
                cropped = img.crop(bbboxes[i+photo_number*2][k])
                cropped.save("result_image/cv_scut"+str(k)+names[i+photo_number*2])
            s_image_path = os.path.join(image_path_root, names[i+photo_number*2])
            viewBBoxes(s_image_path, bboxes, scores)
                             
        elif marks[i]<marks[i+photo_number] and marks[i+photo_number*2]<marks[i+photo_number]:
            bboxes = bbboxes[i+photo_number]
            scores = marks[i+photo_number]
                             
            img = Image.open("example_images/"+names[i+photo_number])
            for k in range(5):
                cropped = img.crop(bbboxes[i+photo_number][k])
                cropped.save("result_image/cv_scut"+str(k)+names[i+photo_number])
            s_image_path = os.path.join(image_path_root, names[i+photo_number])
            viewBBoxes(s_image_path, bboxes, scores)
        

print("DEBUG")

5558 1699 2660 2648


<Figure size 2000x2000 with 5 Axes>

<Figure size 2000x2000 with 5 Axes>

<Figure size 2000x2000 with 5 Axes>

<Figure size 2000x2000 with 5 Axes>

...

<Figure size 2000x2000 with 5 Axes>

<Figure size 2000x2000 with 5 Axes>

<Figure size 2000x2000 with 5 Axes>

<Figure size 2000x2000 with 5 Axes>

KeyboardInterrupt: 